In [1]:
import csv
import math
from string import digits
import pandas as pd

In [58]:
file = '/Users/declanchin/Desktop/MEDSL/2020-precincts/precinct/RI/raw/RI_precincts.csv'

In [59]:
df = pd.read_csv(file)

In [60]:
df.drop(columns='Unnamed: 0', inplace=True)

In [61]:
# uppercasing all columns
df = df.applymap(lambda x:x.upper() if type(x) == str else x)

In [62]:
# turning boolean values into strings
def fix_boolean(x):
    if x == True:
        return 'TRUE'
    else:
        return 'FALSE'
df['special'] = df['special'].apply(fix_boolean)
df['writein'] = df['writein'].apply(fix_boolean)
df['readme_check'] = 'FALSE'

In [63]:
# making certain columns strings
df['state_fips'] = df['state_fips'].astype(str)
df['state_cen'] = df['state_cen'].astype(str)
df['state_ic'] = df['state_ic'].astype(str)
df['county_fips'] = df['county_fips'].apply(lambda x:str(int(x)) if math.isnan(x) == False else '')

In [64]:
def cleanCandidate(x):
    if x == 'ROQUE ROCKY DE LA FUENTE':
        return 'ROQUE "ROCKY" DE LA FUENTE'
    if x == 'WILLIAM W O BRIEN':
        return "WILLIAM W O'BRIEN"
    if x=='WILLIAM R O CONNELL':
        return "WILLIAM R O'CONNELL"
    if x=='WILLIAM M O DELL':
        return "WILLIAM M O'DELL"
    if x=='MARCIA P RANGLIN VASSELL':
        return 'MARCIA P RANGLIN-VASSELL'
    if x=='BARBARA ANN FENTON FUNG':
        return 'BARBARA ANN FENTON-FUNG'
    if x=='CAMILLE F VELLA WILKINSON':
        return 'CAMILLE F VELLA-WILKINSON'
    if x=='RICHARD P D ADDARIO':
        return "RICHARD P D'ADDARIO"
    if x=='TERRI DENISE CORTVRIEND':
        return 'TERRI-DENISE CORTVRIEND'
    if x=='MARGARET M O NEILL':
        return "MARGARET M O'NEILL"
    if x=='CLAIRE VALLIERES O HARA':
        return "CLAIRE VALLIERES O'HARA"
    if x=='JOHN F D AMICO':
        return "JOHN F D'AMICO"
    if x=='SARA K TINDALL WOODMAN':
        return 'SARA K TINDALL-WOODMAN'
    if x=='AUDREY J MACLEOD PFEIFFER':
        return 'AUDREY J MACLEOD-PFEIFFER'
    if x=='LISA BALDELLI HUNT':
        return 'LISA BALDELLI-HUNT'
    if x=='ELIZABETH J COOK MARTIN':
        return 'ELIZABETH J COOK-MARTIN'
    if x=='JEANNE MARIE NAPOLITANO':
        return 'JEANNE-MARIE NAPOLITANO'
    if x == 'WRITE IN':
        return 'WRITEIN'
    if x == 'APPROVE':
        return 'YES'
    if x == 'REJECT':
        return 'NO'
    else:
        return x
df['candidate'] = df['candidate'].apply(cleanCandidate)

In [65]:
df['precinct'] = df['precinct'].str.rstrip()

In [66]:
# merge with jurisdiction fips csv
jurisdiction_fips = pd.read_csv('../../help-files/jurisdiction-fips-codes.csv')
jurisdiction_fips = jurisdiction_fips[jurisdiction_fips['state']=='Rhode Island'].drop(columns='state')
jurisdiction_fips['county_fips'] = jurisdiction_fips['jurisdiction_fips'].apply(lambda fips: str(fips)[:5])
df = df.merge(jurisdiction_fips, on=['jurisdiction_name', 'county_fips'], how='left')
df['jurisdiction_fips'] = df['jurisdiction_fips'].fillna('')
df['jurisdiction_fips'] = df['jurisdiction_fips'].apply(lambda x:str(int(x)) if x != '' else x)

In [67]:
def providenceLimited(x, y):
    if x == 'PROVIDENCE LIMITED':
        return '44007'
    return y
df['jurisdiction_fips'] = df.apply(lambda x: providenceLimited(x['jurisdiction_name'], x['jurisdiction_fips']), axis=1)

In [68]:
def cleanDistrict(x):
    if 'DISTRICT' in str(x):
        dist_num = ''.join([str(n) for n in list(x) if n.isdigit()])
        return str(dist_num).zfill(3)
    else:
        return x
df['district'] = df['district'].apply(cleanDistrict)

In [69]:
def cleanParty(x, y):
    x = x.strip()
    if 'WRITEIN' in y:
        return ''
    if 'DEM' in x:
        return 'DEMOCRAT'
    if 'REP' in x:
        return 'REPUBLICAN'
    if 'S&L' in x:
        return 'PARTY FOR SOCIALISM AND LIBERATION'
    if 'NON' in x:
        return 'NONPARTISAN'
    if 'ALL' in x:
        return 'ALLIANCE'
    if 'AS' in x:
        return 'AMERICAN SOLIDARITY'
    if 'IND' in x:
        return 'INDEPENDENT'
    if 'LIB' in x:
        return 'LIBERTARIAN'
    if 'GRN' in x:
        return 'GREEN'
    else:
        return x
df['party_detailed'] = df.apply(lambda x: cleanParty(x['party_detailed'], x['candidate']), axis=1)
df['party_simplified'] = df['party_detailed'].replace({'PARTY FOR SOCIALISM AND LIBERATION': 'OTHER', 'ALLIANCE': 'OTHER', 'GREEN': 'OTHER', 'AMERICAN SOLIDARITY': 'OTHER', 'INDEPENDENT' : 'OTHER', 'GG' : 'OTHER'})

In [70]:
def cleanOffice(x):
#     x = x.lstrip(digits)
    x = x.lstrip('. ')
    if 'SENATOR IN CONGRESS' in x:
        return 'US SENATE'
    if 'PRESIDENT' in x:
        return 'US PRESIDENT'
    if 'REPRESENTATIVE IN CONGRESS' in x:
        return 'US HOUSE'
    if 'REPRESENTATIVE IN GENERAL ASSEMBLY' in x:
        return 'STATE HOUSE'
    if 'SENATOR IN GENERAL ASSEMBLY' in x:
        return 'STATE SENATE'
    x = x.strip()
    x = x.replace('  ', ' ')
    if x=='TOWN COUNCIL  TOWN OF CUMBERLAND':
        return 'TOWN COUNCIL TOWN OF CUMBERLAND'
    return x
df['office'] = df['office'].apply(cleanOffice)

In [71]:
def cleanDataverse(x, y):
    if any(s in x for s in ['STATE HOUSE', 'STATE SENATE']):
        return 'STATE'
    else:
        return y
df['dataverse'] = df.apply(lambda x: cleanDataverse(x['office'], x['dataverse']), axis=1)

In [72]:
def fixMagnitude(x):
    if any(s in x for s in['TOWN COUNCIL TOWN OF BARRINGTON', 'SCHOOL COMMITTEE TOWN OF BARRINGTON', 
                           'NON-PARTISAN REGIONAL SCHOOL COMMITTEE TOWN OF BRISTOL', 'TOWN COUNCIL TOWN OF BURRILLVILLE',
                           'NON-PARTISAN SCHOOL COMMITTEE TOWN OF BURRILLVILLE', 'COUNCIL - CITY WIDE CITY OF CRANSTON',
                           'SCHOOL COMMITTEE TOWN OF EAST GREENWICH', 'NON-PARTISAN SCHOOL COMMITTEE TOWN OF GLOCESTER',
                           'SCHOOL COMMITTEE TOWN OF JAMESTOWN', 'TOWN COUNCIL TOWN OF NEW SHOREHAM', 'NON-PARTISAN SCHOOL COMMITTEE TOWN OF NEW SHOREHAM',
                           'SCHOOL COMMITTEE TOWN OF NORTH KINGSTOWN', 'NON-PARTISAN SCHOOL COMMITTEE NORTH SMITHFIELD',
                           'COUNCIL AT - LARGE CITY OF PAWTUCKET', 'SCHOOL COMMITTEE TOWN OF PORTSMOUTH',
                           'NON-PARTISAN SCHOOL COMMITTEE TOWN OF TIVERTON']):
        return 3
    if any(s in x for s in['TOWN COUNCIL TOWN OF BRISTOL', 'TOWN COUNCIL TOWN OF CHARLESTOWN', 
                           'TOWN COUNCIL TOWN OF EAST GREENWICH', 'TOWN COUNCIL TOWN OF EXETER',
                           'TOWN COUNCIL TOWN OF FOSTER', 'TOWN COUNCIL TOWN OF GLOCESTER', 'TOWN COUNCIL TOWN OF HOPKINTON',
                           'TOWN COUNCIL TOWN OF JAMESTOWN', 'TOWN COUNCIL TOWN OF LITTLE COMPTON', 'NON-PARTISAN TOWN COUNCIL TOWN OF NARRAGANSETT',
                           'NON-PARTISAN SCHOOL COMMITTEE TOWN OF NARRAGANSETT', 'TOWN COUNCIL TOWN OF NORTH KINGSTOWN',
                           'NON-PARTISAN TOWN COUNCIL TOWN OF NORTH SMITHFIELD', 'TOWN COUNCIL TOWN OF RICHMOND',
                           'TOWN COUNCIL TOWN OF SMITHFIELD', 'TOWN COUNCIL TOWN OF SOUTH KINGSTOWN', 'NON-PARTISAN BUDGET COMMITTEE TOWN OF TIVERTON',
                           'SCHOOL COMMITTEE TOWN OF WESTERLY', 'TOWN COUNCIL TOWN OF WEST GREENWICH', 'NON-PARTISAN SCHOOL COMMITTEE CITY OF WOONSOCKET']):
        return 5
    if any(s in x for s in['NON-PARTISAN CITY COUNCIL - AT LARGE', 'TOWN COUNCIL TOWN OF CUMBERLAND', 'NON-PARTISAN SCHOOL COMMITTEE',
                           'NON-PARTISAN FIRE COMMITTEE TOWN OF CUMBERLAND', 'SCHOOL COMMITTEE TOWN OF EXETER', 
                           'NON-PARTISAN SCHOOL COMMITTEE- TOWN OF LINCOLN', 'SCHOOL COMMITTEE TOWN OF LITTLE COMPTON',
                           'NON-PARTISAN SCHOOL COMMITTEE TOWN OF MIDDLETOWN', 'NON-PARTISAN BLOCK ISLAND LAND TRUST TRUSTEE', 'NON-PARTISAN BLOCK ISLAND HOUSING BOARD',
                           'CHARIHO SCHOOL COMMITTEE TOWN OF RICHMOND', 'SCHOOL COMMITTEE TOWN OF SCITUATE',
                           'SCHOOL COMMITTEE TOWN OF SMITHFIELD', 'TOWN COUNCIL TOWN OF WARREN', 'NON-PARTISAN BRISTOL/WARREN REGIONAL SCHOOL COMMITTEE',
                           'NON-PARTISAN SCHOOL COMMITTEE CITY OF WARWICK']):
        return 2
    if any(s in x for s in['NON-PARTISAN PLANNING COMMISSION TOWN OF CHARLESTOWN', 'NON-PARTISAN COUNCIL- CITY OF NEWPORT',
                           'SCHOOL COMMITTEE TOWN OF SOUTH KINGSTOWN']):
        return 4
    if any(s in x for s in['NON-PARTISAN TOWN COUNCIL TOWN OF MIDDLETOWN', 'NON-PARTISAN SCHOOL COMMITTEE CITY OF NEWPORT',
                           'SCHOOL COMMITTEE CITY OF PAWTUCKET', 'TOWN COUNCIL TOWN OF PORTSMOUTH', 'TOWN COUNCIL TOWN OF SCITUATE',
                           'NON-PARTISAN TOWN COUNCIL TOWN OF TIVERTON', 'TOWN COUNCIL TOWN OF WESTERLY', 
                           'NON-PARTISAN CITY COUNCIL CITY OF WOONSOCKET']):
        return 7
    if any(s in x for s in['NON-PARTISAN CHARTER REVIEW COMMISSION TOWN OF WARREN']):
        return 9
    return 1
df['magnitude'] = df['office'].apply(fixMagnitude)

In [73]:
def getDistrict(x, y):
    if any(s in x for s in ['AT - LARGE', '- AT LARGE']):
        return 'AT-LARGE'
    else:
        return y
df['district'] = df.apply(lambda x: getDistrict(x['office'], x['district']), axis=1)

In [74]:
def standardize_office(x):
    if ' AT - LARGE ' in x:
        x = x.replace(' AT - LARGE ', ' ')
    elif ' - AT LARGE' in x:
        x = x.replace(' - AT LARGE', '')
    if 'NON-PARTISAN ' in x:
        x = x.replace('NON-PARTISAN ', '')
    if x=='SCHOOL COMMITTE EAST PROVIDENCE':
        return 'SCHOOL COMMITTEE - EAST PROVIDENCE'
    if '- CITY OF ' in x:
        x = x.replace('- CITY OF ', ' - ')
        return x
    if '- TOWN OF ' in x:
        x = x.replace('- TOWN OF ', ' - ')
        return x
    if ' TOWN OF ' in x:
        x = x.replace(' TOWN OF ', ' - ')
        return x
    elif 'TOWN COUNCIL ' in x:
        x = 'TOWN COUNCIL - ' + x.split('TOWN COUNCIL ')[1]
        return x
    if ' CITY OF ' in x:
        x = x.replace(' CITY OF ', ' - ')
        return x
    if 'COUNCIL ' in x:
        x = x.replace('COUNCIL ', 'COUNCIL - ')
        return x
    if 'FIRE COMMITTEE ' in x:
        x = x.replace('FIRE COMMITTEE ', 'FIRE COMMITTEE - ')
        return x
    if 'SCHOOL COMMITTEE ' in x:
        if '-' not in x:
            x = x.replace('SCHOOL COMMITTEE ', 'SCHOOL COMMITTEE - ')
            return x
        else:
            return x
    if x=='SCHOOL COMMITTEE':
        return 'SCHOOL COMMITTEE - CUMBERLAND'
    if x=='CITY COUNCIL':
        return 'CITY COUNCIL - CENTRAL FALLS'
    return x
    
df['office'] = df['office'].apply(standardize_office)

In [75]:
column_names = ['precinct', 'office', 'party_detailed', 'party_simplified', 'mode', 'votes', 
                'county_name', 'county_fips', 'jurisdiction_name', 'jurisdiction_fips', 'candidate', 
                'district', 'dataverse', 'year', 'stage', 'state', 'special', 'writein', 'state_po', 
                'state_fips', 'state_cen', 'state_ic', 'date', 'readme_check', 'magnitude']
df = df.reindex(columns=column_names)
df.fillna('', inplace=True)

# # some instances of near duplicate writein scattered rows in raw data. need to agg these votes within precinct and replace to prevent dups
multi_federal_precinct_agg = df[df.drop(columns = 'votes').duplicated(keep = False)].groupby(list(df.drop(columns = 'votes').columns)).agg(sum).reset_index()
# drop near duplicates
df = df[~(df.drop(columns = 'votes').duplicated(keep = False))].copy()
df = pd.concat([df,multi_federal_precinct_agg])

df.to_csv('2020-ri-precinct-general.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

- not sure about "GG" party, tried to find info online but couldn't get anything on the town council position.
- is year supposed to be coded as a string or number?
- some missing county and jurisdiction info

In [22]:
testdf = df

In [23]:
testdf[testdf['jurisdiction_fips']=='nan']

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude


In [26]:
test = pd.read_csv('/Users/sophiazheng/Documents/GitHub/2020-precincts/precinct/RI/2020-ri-precinct-general.csv')
test

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
0,BARRINGTON 0101,US PRESIDENT,REPUBLICAN,REPUBLICAN,ELECTION DAY,229,BRISTOL,44001.0,BARRINGTON,4.400105e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,1
1,BARRINGTON 0104,"COUNCIL - NUMBER, SELECTION, TERM - BARRINGTON",NONPARTISAN,NONPARTISAN,ELECTION DAY,120,BRISTOL,44001.0,BARRINGTON,4.400105e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,1
2,BARRINGTON 0105,TOWN COUNCIL - BARRINGTON,DEMOCRAT,DEMOCRAT,ELECTION DAY,129,BRISTOL,44001.0,BARRINGTON,4.400105e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,3
3,BARRINGTON LIMITED,US HOUSE,NaN,NaN,ELECTION DAY,0,BRISTOL,44001.0,BARRINGTON,4.400105e+09,...,RHODE ISLAND,False,True,RI,44,15,5,2020-11-03,False,1
4,BARRINGTON 0105,US SENATE,REPUBLICAN,REPUBLICAN,ELECTION DAY,187,BRISTOL,44001.0,BARRINGTON,4.400105e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49714,WOONSOCKET 3902,SCHOOL COMMITTEE - WOONSOCKET,NONPARTISAN,NONPARTISAN,EARLY VOTING,53,PROVIDENCE,44007.0,WOONSOCKET,4.400781e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,5
49715,WOONSOCKET 3903,SCHOOL COMMITTEE - WOONSOCKET,NONPARTISAN,NONPARTISAN,EARLY VOTING,49,PROVIDENCE,44007.0,WOONSOCKET,4.400781e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,5
49716,WOONSOCKET 3904,SCHOOL COMMITTEE - WOONSOCKET,NONPARTISAN,NONPARTISAN,EARLY VOTING,108,PROVIDENCE,44007.0,WOONSOCKET,4.400781e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,5
49717,WOONSOCKET 3905,SCHOOL COMMITTEE - WOONSOCKET,NONPARTISAN,NONPARTISAN,EARLY VOTING,83,PROVIDENCE,44007.0,WOONSOCKET,4.400781e+09,...,RHODE ISLAND,False,False,RI,44,15,5,2020-11-03,False,5


In [27]:
test[test['jurisdiction_fips']=='nan']

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
